In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [14]:
train_df = pd.read_csv(r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\data\train_processed.csv")

In [15]:
print(train_df.columns.tolist())


['sample_id', 'catalog_content', 'image_link', 'price', 'item_name', 'bullet_points', 'product_description', 'value', 'unit', 'quantity_standard', 'unit_standard', 'quantity_numeric', 'quantity_count', 'price_per_unit']


In [16]:
# Numeric
numeric_cols = ['value', 'quantity_numeric', 'quantity_count', 'price_per_unit']

# Categorical
categorical_cols = ['unit', 'unit_standard']  # you can add more if needed

# Target
y_train = train_df['price'].values

In [17]:
numeric_data = train_df[numeric_cols].values
scaler = StandardScaler()
numeric_data_scaled = scaler.fit_transform(numeric_data)


In [19]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_data_encoded = ohe.fit_transform(train_df[categorical_cols])


In [21]:
train_cnn = np.load(r"C:\Users\Ekaansh\OneDrive\Desktop\AB\vs code\JS\projects\hackathon\amazon\models\cnn_features_efficientnetb3_torch.npy")  # shape (75000, 1536)
pca_components = 1024
pca = PCA(n_components=pca_components)
train_cnn_reduced = pca.fit_transform(train_cnn)

In [22]:
X_train_full = np.hstack([numeric_data_scaled, cat_data_encoded, train_cnn_reduced])
print("Final feature shape:", X_train_full.shape)

Final feature shape: (75000, 1117)


In [23]:
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_full, y_train, test_size=0.2, random_state=42
)
print(f"Train shape: {X_tr.shape}, Validation shape: {X_val.shape}")

Train shape: (60000, 1117), Validation shape: (15000, 1117)


In [24]:
dtrain = xgb.DMatrix(X_tr, label=y_tr)
dval = xgb.DMatrix(X_val, label=y_val)

In [25]:
params = {
    'objective': 'reg:squarederror',
    'tree_method': 'gpu_hist',      # GPU acceleration
    'predictor': 'gpu_predictor',
    'learning_rate': 0.05,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'eval_metric': 'rmse'
}

evallist = [(dval, 'eval'), (dtrain, 'train')]
num_round = 1000

In [26]:
bst = xgb.train(
    params,
    dtrain,
    num_boost_round=num_round,
    evals=evallist,
    early_stopping_rounds=50,
    verbose_eval=50  # logs every 50 rounds
)

c:\Users\Ekaansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\callback.py:386: UserWarning: [12:24:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  self.starting_round = model.num_boosted_rounds()
c:\Users\Ekaansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\callback.py:386: UserWarning: [12:24:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor" } are not used.

  self.starting_round = model.num_boosted_rounds()


[0]	eval-rmse:38.30758	train-rmse:31.15447
[50]	eval-rmse:30.18747	train-rmse:18.60616
[100]	eval-rmse:29.12714	train-rmse:15.48025
[150]	eval-rmse:28.90168	train-rmse:13.70894
[200]	eval-rmse:28.76246	train-rmse:12.47195
[250]	eval-rmse:28.69253	train-rmse:11.57196
[300]	eval-rmse:28.61780	train-rmse:10.76271
[350]	eval-rmse:28.57077	train-rmse:9.98857
[400]	eval-rmse:28.51975	train-rmse:9.32266
[450]	eval-rmse:28.49493	train-rmse:8.73186
[500]	eval-rmse:28.47966	train-rmse:8.21145
[550]	eval-rmse:28.44343	train-rmse:7.70899
[600]	eval-rmse:28.42572	train-rmse:7.24993
[650]	eval-rmse:28.42292	train-rmse:6.81848
[700]	eval-rmse:28.41765	train-rmse:6.40781
[750]	eval-rmse:28.39340	train-rmse:6.00942
[800]	eval-rmse:28.38928	train-rmse:5.66102
[850]	eval-rmse:28.37710	train-rmse:5.34032
[900]	eval-rmse:28.37567	train-rmse:5.04625
[950]	eval-rmse:28.37178	train-rmse:4.77530
[999]	eval-rmse:28.36988	train-rmse:4.52205


In [31]:
from sklearn.metrics import mean_squared_error


In [33]:
y_val_pred = bst.predict(dval)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print("✅ Validation RMSE with CNN + numeric + categorical features:", rmse)

✅ Validation RMSE with CNN + numeric + categorical features: 28.369875582931556


In [34]:
import joblib
joblib.dump(bst, "xgb_model_with_cnn_numeric_cat.pkl")
print("💾 Model saved as xgb_model_with_cnn_numeric_cat.pkl")

💾 Model saved as xgb_model_with_cnn_numeric_cat.pkl


In [35]:
import numpy as np

def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# Example:
smape_val = smape(y_val, y_val_pred)
print(f"✅ Validation SMAPE: {smape_val:.4f}%")


✅ Validation SMAPE: 26.4200%
